<H3>Create new DB</H3>

In [39]:
%%writefile creation_db.hql

DROP DATABASE IF EXISTS af_demodb CASCADE;

CREATE DATABASE af_demodb LOCATION '/user/jovyan/af_store';

Writing creation_db.hql


In [40]:
! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.055 seconds
OK
Time taken: 0.458 seconds


<H3>Parse data from HDFS location '/data/stackexchange1000/posts' into table posts_sample_external using the regex expression</H3>

In [41]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE af_demodb;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(row_id int,
post_type_id int,
year int,
month int)
ROW FORMAT 
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = ".*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*$"
)
LOCATION '/data/stackexchange1000/posts';

Overwriting query.hql


In [42]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.039 seconds
OK
Time taken: 0.102 seconds
OK
Time taken: 0.866 seconds


<h3>Check the content of the table posts_sample_external</h3>
(note the presense of NULL rows)

In [43]:
! hive --database af_demodb -e 'select * from posts_sample_external limit 5;'


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.086 seconds
OK
NULL	NULL	NULL	NULL
1394	2	2008	8
3543	2	2008	8
4521	2	2008	8
8689	2	2008	8
Time taken: 1.706 seconds, Fetched: 5 row(s)


<h3> Create the table posts_sample partitioned by year and month within HDFS location '/user/jovyan/af_store/' </h3>

In [53]:
%%writefile query2.hql

USE af_demodb;

DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string) 
LOCATION '/user/jovyan/af_store/';

Overwriting query2.hql


In [54]:
! hive -f query2.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.136 seconds
OK
Time taken: 1.727 seconds
OK
Time taken: 0.688 seconds


<h3>SELECT query to form try and form output as requested by the assignment (year 'tab' month 'table' lines count)</h3>

In [75]:
%%writefile query_try.hql

USE af_demodb;

FROM posts_sample_external
SELECT year, concat(year,"-",month) as month, count(row_id)
WHERE year IS NOT NULL
GROUP BY year, month;

Overwriting query_try.hql


In [76]:
! hive -f query_try.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.176 seconds
Query ID = jovyan_20180512023131_2081d5c9-4207-4c3d-9af2-b8002c397267
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526086425574_0007, Tracking URL = http://fb266a9b2d9d:8088/proxy/application_1526086425574_0007/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526086425574_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-12 02:31:55,401 Stage-1 map = 0%,  reduce = 0%
2018-05-12 02:32:13,784 Stage-1 map = 39%,  

<h3> Populate 'posts_sample' table with data from 'posts_sample_external' table </h3>

In [78]:
%%writefile query3.hql

set hive.exec.dynamic.partition.mode=nonstrict;

USE af_demodb;

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT year, concat(year,"-",month) as month, count(row_id) as count
WHERE year IS NOT NULL
GROUP BY year, month;

Overwriting query3.hql


In [79]:
! hive -f query3.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.16 seconds
Query ID = jovyan_20180512023535_0a565c6f-6f07-445c-91bb-69bacb977687
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526086425574_0008, Tracking URL = http://fb266a9b2d9d:8088/proxy/application_1526086425574_0008/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526086425574_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-12 02:35:54,340 Stage-1 map = 0%,  reduce = 0%
2018-05-12 02:36:12,777 Stage-1 map = 36%,  r

Partition af_demodb.posts_sample{year=2011-8, month=362} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2011-9, month=343} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-1, month=377} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-10, month=469} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-11, month=438} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-12, month=415} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-2, month=412} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-3, month=432} stats: [numFiles=1, numRows=1, totalSize=5, rawDataSize=4]
Partition af_demodb.posts_sample{year=2012-4, month=412} stats: [numF

<h3> get the line for "2008-10"</h3>

In [88]:
%%writefile query4.hql

USE af_demodb;

SELECT year, month, count 
FROM posts_sample 
WHERE month = '2008-10';

Overwriting query4.hql


In [89]:
! hive -f query4.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.128 seconds
OK
2008	2008-10	73
Time taken: 2.73 seconds, Fetched: 1 row(s)
